<a href="https://colab.research.google.com/github/ayushigupta123/Data-Analysis-/blob/main/Titanic_survivor_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Titanic survivor analysis***


---

Using ML in data analysis finding more about survival of passengers in Titanic based on gender, rich/ordinary class, with or without family and place from which they boarded ship





Importing pandas and numpy python libraries

In [34]:
import pandas as pd
import numpy as np

pd.DataFrame() is used to get values from file i.e. get dagta from pandas

In [35]:
df = pd.DataFrame(pd.read_csv('train.csv'))
df.head() 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


df.shape is used to tell how many rows and columns are there in dataset

In [36]:
df.shape


(891, 12)

Finding sum of null values present in column 

In [37]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Separting out columns which have more than 35% of values mising

In [38]:
drop_col = df.isnull().sum()[df.isnull().sum()>(35/100 * df.shape[0])]
drop_col

Cabin    687
dtype: int64

Finding index of drop_col

In [39]:
drop_col.index

Index(['Cabin'], dtype='object')

Dropping a column from entire dataset

In [40]:
df.drop(drop_col.index, axis =1, inplace=True)
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

Getting meann values where values are not String

In [41]:
df.fillna(df.mean(), inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Embarked contain string values which does not have mean.

In [42]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For Embarked attribute, we fill null values with most frequent value in column

In [43]:
df['Embarked'].fillna('S', inplace=True)

In [44]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

corr() - corelation between different columns

In [45]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


Making new column 'family' from 'sibsp' and 'parch'
Dropping columns used to make new column to remove redundancy

In [46]:
df['FamilySize']=df['SibSp']+df['Parch']
df.drop(['SibSp', 'Parch'],axis=1, inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


check if person was alone or not can effect survival rate
head() gets first 5 rows

In [47]:
df['Alone']= [0 if df['FamilySize'][i]>0 else 1 for i in df.index]
df.head

<bound method NDFrame.head of      PassengerId  Survived  Pclass  ... Embarked FamilySize  Alone
0              1         0       3  ...        S          1      0
1              2         1       1  ...        C          1      0
2              3         1       3  ...        S          0      1
3              4         1       1  ...        S          1      0
4              5         0       3  ...        S          0      1
..           ...       ...     ...  ...      ...        ...    ...
886          887         0       2  ...        S          0      1
887          888         1       1  ...        S          0      1
888          889         0       3  ...        S          3      0
889          890         1       1  ...        C          0      1
890          891         0       3  ...        Q          0      1

[891 rows x 11 columns]>

Find mean of person survived whether alone or not

In [48]:
df.groupby(['Alone'])['Survived'].mean()


Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

finding corelation between aloe and fare. 

In [49]:
df[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


Finding if number of female survived is more than male

In [50]:
df['Sex']= [0 if df['Sex'][i]=='male' else 1 for i in df.index] #1 for female 0 for male
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

Finding percentage of people survived embarked i.e. boarded from particular place

In [ ]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

So, according to data analysis we found that:


*   More female survived than male
*   Rich people have more survival rate
*   Passenger with family has higher survival rate
*   Passenger who boarded ship from c have more survival rate





